In [4]:
import random

def generateBoard(cols, rows, mines):
    board = [[0 for x in range(cols)] for y in range(rows)]

    #generate mine locations
    minePos = random.sample(range(rows * cols), mines)
    for pos in minePos:
        row, col = divmod(pos, cols)
        board[row][col] = 'M'

    #count neighbors
    for row in range(rows):
        for col in range(cols):
            if board[row][col] == 'M':
                continue
            count = 0
            for rowShift in [-1, 0, 1]:
                for colShift in [-1, 0, 1]:
                    r= row + rowShift
                    c = col + colShift
                    if (0 <= r < rows) and (0 <= c < cols) and (board[r][c] == 'M'):
                        count += 1
            board[row][col] = count

    return board

def generatePlayArea(cols, rows):
    PlayArea = [['X' for x in range(cols)] for y in range(rows)]
    return PlayArea

def checkSpot(board, col, row):
    result = board[row][col]
    return result

In [9]:
board = generateBoard(30,16,40)
playArea = generatePlayArea(30,16)

for row in board:
    print(row)
print("----------")
for row in playArea:
    print(row)

[0, 1, 'M', 1, 1, 'M', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 'M', 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 'M', 1, 0, 2, 2, 2, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 'M', 2, 'M', 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 'M', 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 'M', 1, 1, 1, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0]
[0, 0, 1, 1, 1, 0, 0, 0, 1, 2, 2, 2, 1, 1, 1, 'M', 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 'M', 1, 0]
[0, 0, 1, 'M', 1, 0, 0, 1, 1, 2, 'M', 3, 'M', 2, 1, 2, 'M', 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0]
[0, 0, 1, 1, 2, 1, 1, 1, 'M', 2, 2, 4, 'M', 2, 0, 1, 1, 1, 1, 'M', 1, 0, 1, 'M', 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2, 'M', 2, 1, 1, 1, 2, 'M', 3, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 2, 'M', 2, 0, 0, 0, 2, 'M', 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 'M', 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 'M', 1, 0, 0, 1, 1, 1]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
import copy
import numpy as np

def countNeighborUnknowns(playArea, col, row):
    unseenCount = 0
    flagCount = 0
    for colShift in [-1, 0, 1]:
        for rowShift in [-1, 0, 1]:
            curCol = col + colShift
            curRow = row + rowShift

            if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0])):
                if(playArea[curRow][curCol] == 'X'):
                    unseenCount += 1
                if(playArea[curRow][curCol] == 'F'):
                    flagCount += 1

    return unseenCount, flagCount



def getFrontier(playArea):
    frontier = []
    for i in range(len(playArea)):
        for j in range(len(playArea[0])):
            unseenCount, flagCount = countNeighborUnknowns(playArea, j, i)
            if (isinstance(playArea[i][j], int) and unseenCount >= 1):

                #could append both unseen count or flag count here?
                frontier.append((j,i, unseenCount, flagCount))

    return frontier

def depthFirstSearch(board, playArea, col, row):
    if (row < 0 or row >= len(board) or col < 0 or col >= len(board[0]) or playArea[row][col] != 'X'):
        return

    playArea[row][col] = checkSpot(board, col, row)

    if playArea[row][col] == 'M':
        print("lost during dfs")
        return

    # if no nearby mines, check all nearby squares
    if playArea[row][col] == 0:
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                depthFirstSearch(board, playArea, col+i, row+i)
    elif isinstance(playArea[row][col], int):
        # if it is a number, we need to backtrack?
        return

def solveCertainMoves(board, playArea):
    frontier = getFrontier(playArea)
    for (col, row, unseenCount, flagCount) in frontier:
        if (playArea[row][col] == 'X' or playArea[row][col] == 'F'):
            print("string in the frontier")
            continue
        if (playArea[row][col] - flagCount == unseenCount):
            for colShift in [-1, 0, 1]:
                for rowShift in [-1, 0, 1]:
                    curCol = col + colShift
                    curRow = row + rowShift

                    if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                        playArea[curRow][curCol] = 'F'
        elif (playArea[row][col] - flagCount == 0 and unseenCount > 0):
            # if all mines have been seen, but there are still more neighbor squares to explore
            for colShift in [-1, 0, 1]:
                for rowShift in [-1, 0, 1]:
                    curCol = col + colShift
                    curRow = row + rowShift

                    if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                        depthFirstSearch(board, playArea, curCol, curRow)

def guess(board, playArea):
    frontier = getFrontier(playArea)
    bestOdd = 1.1
    bestCellRow = -1
    bestCellCol = -1

    #iterate across frontier, getting cell with best odds and guess that one of it's adjacent neighbors is not a mine
    for (col, row, unseenCount, flagCount) in frontier:
        curVal = playArea[row][col]
        remaining = curVal - flagCount
        curOdds = remaining / unseenCount
        if (curOdds < bestOdd):
            bestOdd = curOdds
            bestCellCol = col
            bestCellRow = row
    
    #get an adjacent unexplored
    for colShift in [-1, 0, 1]:
        for rowShift in [-1, 0, 1]:
            curCol = bestCellCol + colShift
            curRow = bestCellRow + rowShift
            if (curRow >= 0 and curRow < len(playArea) and curCol >= 0 and curCol < len(playArea[0]) and playArea[curRow][curCol] == 'X'):
                guessResult = checkSpot(board, curCol, curRow)
                print(f"Made guess with odds: {bestOdd:.4f} on cell: ({curCol}, {curRow})")
                playArea[curRow][curCol] = guessResult
                if (guessResult == 'M'):
                    print("Lost in guess function")
                    return False
                else:
                    print("Guess succeeds")
                    return True
    
    print("didn't find adjacent unexplored during guess")
    return False


def checkIfDone(playArea):
    for row in range(len(playArea)):
        for col in range(len(playArea[0])):
            if (playArea[row][col] == 'X' or playArea[row][col] == 'M'):
                return False
    return True



def solve(board, playArea):
    isAlive = True
    pastState = copy.deepcopy(playArea)
    row = -1
    col = -1
    foundStart = False
    
    for i in range(len(board)):
        for j in range(len(board[0])):
            cellVal = checkSpot(board, col, row)

            if cellVal == 'M':
                #print("lost during solve")
                continue
            else:
                row = i
                col = j
                foundStart = True
                break
        if foundStart:
            break
    #explore zeros
    depthFirstSearch(board, playArea, col, row)

    while (not checkIfDone(playArea)):
        #create frontier
        #solve certain moves
        solveCertainMoves(board, playArea)

        #check if past state is same as cur state
        # if so, we need to guess
        np_playArea = np.array(playArea)
        np_pastState = np.array(pastState)
        if (np.all(np_playArea == np_pastState)):
            isAlive = guess(board, playArea)

        pastState = copy.deepcopy(playArea)

        if (not isAlive):
            print("dead")
            return False
    
    print("win")
    return True


board = generateBoard(30,16,70)
playArea = generatePlayArea(30,16)

for row in playArea:
    print(row)

solve(board, playArea)

print("----------------")

for row in playArea:
    print(row)

['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'

In [35]:
winCount = 0
numGames = 100

for i in range(numGames):
    board = generateBoard(30,16,40)
    playArea = generatePlayArea(30,16)
    result = solve(board, playArea)
    if(result):
        winCount += 1

print(f"Result after {numGames}: {winCount}")

win
Made guess with odds: 0.3333 on cell: (0, 1)
Lost in guess function
dead
win
win
Made guess with odds: 0.5000 on cell: (0, 14)
Guess succeeds
win
win
Made guess with odds: 0.3333 on cell: (0, 1)
Guess succeeds
Made guess with odds: 0.2500 on cell: (0, 2)
Guess succeeds
Made guess with odds: 0.3333 on cell: (1, 0)
Guess succeeds
win
Made guess with odds: 0.3333 on cell: (4, 0)
Guess succeeds
win
win
win
Made guess with odds: 0.3333 on cell: (0, 2)
Guess succeeds
win
win
win
win
win
win
win
win
win
Made guess with odds: 0.3333 on cell: (0, 1)
Guess succeeds
Made guess with odds: 0.2500 on cell: (0, 2)
Guess succeeds
Made guess with odds: 0.2500 on cell: (0, 3)
Guess succeeds
win
win
win
win
win
Made guess with odds: 1.1000 on cell: (0, 0)
Guess succeeds
Made guess with odds: 0.3333 on cell: (0, 2)
Guess succeeds
win
lost during dfs
didn't find adjacent unexplored during guess
dead
win
Made guess with odds: 0.3333 on cell: (0, 1)
Guess succeeds
Made guess with odds: 0.2500 on cell: (0

In [5]:
# class used to store the information of a cluster of cells found by the solver
class Cluster():
    def __init__(self, cells, count):
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        return self.cells == other.cells and self.count == other.count

    # used to 
    def known_mines(self):
        if len(self.cells)==self.count:
            return self.cells
        else:
            return set()
        
    def known_safes(self):

        if self.count==0:
            return self.cells
        else:
            return set()

    def mark_mine(self, cell):
        if cell in self.cells:
            self.cells.discard(cell)
            self.count-=1

    def mark_safe(self, cell):
        if cell in self.cells:
            self.cells.discard(cell)

class Solver():
    def __init__(self, board, play_area, cols, rows):
        self.board = board
        self.play_area = play_area
        self.cols = cols
        self.rows = rows

        self.explored = set()

        self.mines = set()
        self.safes = set()
        self.kb = []

        # set of all possible moves in the board; used for comparison later on
        self.legal_moves = set()
        for i in range(rows):
            for j in range(cols):
                self.legal_moves.add((i,j))

    # used to make a move based on the knowledge base
    # if a move can be deemed safe and has not been explored, return it
    # otherwise, return None
    def make_move(self):
        for move in self.safes:
            if move not in self.explored:
                self.explored.add(move)
                return move
        return None

    # used to make a move when no safe move can be made
    # if a move exists that has not been explored and has not been deemed a mine, return it
    # otherwise, return None
    def make_guess(self):
        for i in range(rows):
            for j in range(cols):
                if((i,j) not in self.explored) and ((i,j)not in self.mines):
                    self.explored.add((i,j))
                    return (i,j)
        return None
    
    # used to mark a cell as a mine
    # calls function existing inside the cluster
    def mark_mine(self, cell):
        self.mines.add(cell)
        for cluster in self.kb:
            cluster.mark_mine(cell)

    # used to mark a cell as safe
    # calls function existing inside the cluster
    def mark_safe(self, cell):
        self.safes.add(cell)
        for cluster in self.kb:
            cluster.mark_safe(cell)

    def update_kb(self, move):
        # mark the move made as one that is safe since it bypassed checks
        self.explored.add(move)
        self.safes.add(move)

        # get cluster of cells that are adjacent to the move made
        cells={(move[0],move[1]-1),(move[0],move[1]+1),(move[0]+1,move[1]),(move[0]-1,move[1]),(move[0]+1,move[1]+1),(move[0]-1,move[1]-1),(move[0]+1,move[1]-1),(move[0]-1,move[1]+1)}
        # remove cells that are out of bounds
        cells = cells.intersection(self.legal_moves)
        # remove cells that are already passed
        cells = cells.difference(self.explored)
        # add the new cluster to the knowledge base
        self.kb.append(Cluster(cells=cells, count=board[move[0]][move[1]]))
        while True:
            # loop through the existing clusters of cells in the knowledge base to find out existing knowledge and try to make new discoveries
            wkb=copy.deepcopy(self.kb)
            for cluster in wkb:
                safe=cluster.known_safes()
                mine=cluster.known_mines()
                if len(safe): # check for new safe cells
                    self.safes.update(safe)
                    self.kb.remove(cluster)
                if len(mine): # check for new mine cells
                    self.mines.update(mine)
                    self.kb.remove(cluster)

            # re-mark mines to make sure there is full coverage based on new info
            for s in self.safes:
                self.mark_safe(s)
            for m in self.mines:
                self.mark_mine(m)

            # find new clusters given the current knowledge base and the discoveries made above
            wkb2 = copy.deepcopy(self.kb)
            for c1 in wkb2:
                for c2 in wkb2:
                    newcells = c2.cells.difference(c1.cells)
                    newcluster = Cluster(cells=newcells,count=(c2.count-c1.count))
                    if not(c2 == c1) and c1.cells.issubset(c2.cells) and (newcluster not in self.kb):
                        self.kb.append(newcluster)
            if(wkb==self.kb):
                break

        # account for if the move was blank and spread to other cells
        if self.play_area[move[0]][move[1]] == ' ':
            for i in range(rows):
                for j in range(cols):
                    if self.play_area[i][j] != 'X' and (i, j) not in self.explored and self.board[i][j] != 'M':
                        self.update_kb((i,j))
    
    def solve(self):
        move = (0,0) # the most optimal starting spot is in the corner, use top left for ease of use
        while (not reveal_cell(self.board, self.play_area, move[1], move[0])) and (not check_win(self.board, self.play_area)):
            # handle the update of the AI model for the next move
            print_board(self.play_area)
            self.update_kb(move)

            # handle choosing the next move for the AI
            # None means that there are no logical steps to take in that route, so when both options return None, the game has been solved
            move = self.make_move()
            if move is None:
                move = self.make_guess()
                if move is None:
                    break
            
        # check whether game was won or lost
        if check_win(self.board, self.play_area):
            print_board(self.play_area)
            print("Board solved :)")
        else:
            print_board(self.play_area)
            print("Mine has been hit :(")